In [2]:
import sol_split_cyl_coarse as RH
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
import scipy as sp
import scipy.sparse.linalg
import time


Implementation of the solution splitting technique with cylindrical symmetry around the vessel:

$$-D \Delta \phi = - \delta_\Lambda q$$

$$\phi \mathbf(x)=0 \quad on \quad \partial \Omega_{ext}$$
$$\nabla_n \phi\mathbf(x)=0 \quad on \quad \partial \Omega_{east, west}$$

$$\iiint_\Omega \delta_\Lambda f\mathbf(x) dV = \int_\Lambda f(s)ds$$

$$\phi= \phi_s + \phi_r$$

$\phi$ represents the scalar field that we want to recover 

$$\phi_s=G(r)q(s)$$

with: $G(r)=-\frac{1}{2 \pi D}log(\frac{r}{R_v})$



$$\Delta \phi = \Delta (G(r)q(s)) + \Delta \phi_r$$
$$-\delta_\Lambda \frac{q}{D} = (\Delta G(r))q(s) + 2\nabla G \cdot \nabla q + Gq"(s) +  \Delta \phi_r$$

For this particular problem we have:
$$\Delta \phi_r = -G(r)\frac{d^2 q}{d s^2}$$

with the following boundary conditions:

$$\frac{\partial \phi_r}{\partial s}= - G(r)\frac{\partial q(s)}{\partial s} \quad  on \quad \partial \Omega_{east,west}$$

$$\phi_r(R_{max}, s) = -G(R_{max})q(s) \quad on \quad \partial \Omega_{ext}$$

For the internal boundary conditions (near the vessel wall:

$$\nabla \phi \cdot \mathbf{n} = - \frac{q(s)}{2 \pi R_v}$$
in this case $\mathbf{n}= - \mathbf{e}_r$ 

Therefore:
$$\frac{\partial \phi}{\partial r}= \frac{q(s)}{2 \pi R_v} \quad on \quad \Omega_{in}$$

Further:
$$\nabla(G(r)q(s))\cdot (-\mathbf{e_r})=\frac{q(s)}{2 \pi R_v}$$

Therefore:
$$\frac{\partial \phi_r}{\partial r}= 0 \quad on \quad \Omega_{in}$$


We obtain the full problem:
\begin{cases}
\Delta \phi_r = -G(r)\dfrac{d^2 q}{d s^2} \\
\\
  \dfrac{\partial \phi_r}{\partial s}= - G(r)\dfrac{\partial q(s)}{\partial s} \quad  on \quad \partial \Omega_{east,west} \\
  \\
  \phi_r(R_{max}, s) = -G(R_{max})q(s) \quad on \quad \partial \Omega_{ext} \\
  \\
  \dfrac{\partial \phi}{\partial r}= 0 \quad on \quad \Omega_{in} \\
\end{cases}  


In [4]:
from importlib import reload
reload(RH)
plt.figure()
plt.plot(RH.k.s,RH.q_vessel)
plt.title("Source given by vessel")
plt.xlabel("s")


NameError: name 'RH' is not defined

<Figure size 504x504 with 0 Axes>

In [5]:
RH.my_west_modification=True
print("inner mode: ",RH.inner_modification)
print("west mode: ", RH.west_mode)
print("My west mode: ", RH.my_west_mode)

NameError: name 'RH' is not defined

In [ ]:

RH.compare_full_solutions(RH.rhs,  RH.r_rhs,"rhs", "real_rhs",[-1,0.8], "Contour RHSs")
rhs_forlater=RH.rhs

In [ ]:
RH.plot_profile(5, RH.k.s, RH.k.r, RH.rhs, RH.r_rhs, "RHS real vs v")

In [ ]:
RH.s_plot_profile(0, RH.k.s, RH.k.r, RH.rhs, RH.r_rhs,"RHS real vs v")

In [ ]:
RH.plot_profile(-1, RH.k.s, RH.k.r, RH.rhs, RH.r_rhs,"RHS real vs v")

In the following code, the vessel containing cells and the west boundaries are modified to equal the "retro_engineered" right hand side. After this two operations are done we continue 

In [ ]:
#RH.rhs[:,-1]=RH.r_rhs[:,-1]
RH.rhs[0,:]=RH.r_rhs[0,:]
RHS=np.ndarray.flatten(RH.rhs)

In [ ]:
def plott(sol, title, cmin, cmax):
    levels=np.linspace(cmin, cmax,20)
    plt.contourf(sol, levels)
    plt.ylabel("r")
    plt.xlabel("s")
    plt.title(title)
    plt.colorbar()

In [ ]:
sol=sp.sparse.linalg.spsolve(RH.Lap_operator, RHS).reshape(len(RH.k.r), len(RH.k.s))
sol_plus=sol+RH.sol_sing
plott(RH.real_v, "modified solution", -0.1, 0.5)


In [ ]:
RH.compare_full_solutions(sol_plus,  RH.sol_real,"split_modified", "real_sol",[0,0.6], "Contour modified")

In [ ]:
RH.compare_full_solutions(RH.rhs,  RH.r_rhs,"rhs", "real_rhs",[-1,0.8], "Contour RHSs")

In [ ]:
RH.r_rhs[-1,:]


In [ ]:
first_der=RH.get_first_der(RH.q_vessel,RH.k.inc_s).dot(RH.q_vessel)
plt.plot(RH.k.s, first_der, label="first derivative vessel")
plt.legend()

In [ ]:
plt.figure()
plt.plot(RH.k.r, RH.Green(RH.k.r, RH.k.Rv), label="Green function")
plt.plot(RH.k.r, -RH.Green(RH.k.r, RH.k.Rv)*RH.q_first[-1], label="boundary value")
plt.legend()
plt.xlabel("r")

In [ ]:
f=plt.figure(figsize=(10,20))

# Create two subplots and unpack the output array immediately
f, (ax1, ax2, ax3) = plt.subplots(1, 3,figsize=(12,8) )
ax1.plot(RH.k.r[1:-1], RH.rhs[1:-1,-1])
ax1.set_title("computed boundary value RHS")
ax2.plot(RH.k.r[1:-1], RH.r_rhs[1:-1,-1])
ax2.set_title("real boundary value RHS")
ax3.plot(RH.k.r[1:-1], RH.r_rhs[1:-1,-1]/RH.rhs[1:-1,-1])
ax3.set_title("ratio")

In [ ]:
1/RH.hs

Fully modified:



In [ ]:
RH.rhs[1:-1,-1]*=-1/RH.hs
RH.rhs[0,:]=RH.r_rhs[0,:]
RHS=np.ndarray.flatten(RH.rhs)
sol=sp.sparse.linalg.spsolve(RH.Lap_operator, RHS).reshape(len(RH.k.r), len(RH.k.s))
sol_plus=sol+RH.sol_sing
RH.plott(sol_plus, "modified solution", -0.1,0.9)
RH.compare_full_solutions(sol_plus,  RH.sol_real,"split_modified", "real_sol",[0,1], "Contour modified")

In [ ]:
RH.rhs=rhs_forlater

In [ ]:
RHS=np.ndarray.flatten(RH.rhs)
sol=sp.sparse.linalg.spsolve(RH.Lap_operator, RHS).reshape(len(RH.k.r), len(RH.k.z))
sol_plus=sol+RH.sol_sing
RH.plott(sol_plus, "modified solution")
RH.compare_full_solutions(sol_plus,  RH.sol_real,"split_modified", "real_sol",[0,1], "Contour modified")